# El propósito de este Notebook es analizar la base de datos de los delitos para determinar cuáles son las colonias más peligrosas, entre otras cosas.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
data = pd.read_csv('delitos_limpio_v1.csv')
data = data.drop(columns=['Unnamed: 0', 'index'])

In [3]:
data.head(4)

,año_hechos,mes_hechos,fecha_hechos,delito,categoria_delito,colonia_hechos,alcaldia_hechos,calle_hechos,longitud,latitud,geo_punto
0,2018.0,Abril,2018-04-13 13:30:00,ROBO DE DOCUMENTOS,DELITO DE BAJO IMPACTO,POLANCO,MIGUEL HIDALGO,MOLIERE,-99.201358,19.436043,"{'type': 'Point', 'coordinates': [-99.20135815..."
1,2016.0,Agosto,2016-08-05 16:30:00,ROBO A TRANSEUNTE EN VIA PUBLICA CON VIOLENCIA,ROBO A TRANSEUNTE EN VÍA PÚBLICA CON Y SIN VIO...,PLAZA COAPA,TLALPAN,AVENIDA CANAL DE MIRAMONTES,-99.125250,19.295906,"{'type': 'Point', 'coordinates': [-99.12524997..."
2,2015.0,Octubre,2015-10-23 12:00:00,FRAUDE,DELITO DE BAJO IMPACTO,DOCTORES,CUAUHTEMOC,AV. NIÑOS HEROES,-99.150120,19.421230,"{'type': 'Point', 'coordinates': [-99.15011999..."
3,2016.0,Marzo,2016-03-28 09:00:00,VIOLENCIA FAMILIAR,DELITO DE BAJO IMPACTO,AMPLIACIÓN CARACOL,VENUSTIANO CARRANZA,GRAL. FRANCISCO MURGUIA,-99.066132,19.428401,"{'type': 'Point', 'coordinates': [-99.06613199..."


# Paso 1: Observar cuáles son las distintas categorías de delitos y agruparlas en 3 niveles: delito menor, delito medio y delito grave. Pronto se verá claro a qué me refiero. 

Para los propósitos de este análisis, no necesito por el momento saber en qué calle fueron los hechos, ni la latitud, longitud o el geopunto, por lo que descartaré esas columnas. 

In [4]:
data = data.drop(columns=['calle_hechos', 'longitud', 'latitud', 'geo_punto'])

In [5]:
#Veamos las categorías de los delitos
print(data.categoria_delito.unique())

['DELITO DE BAJO IMPACTO'
 'ROBO A TRANSEUNTE EN VÍA PÚBLICA CON Y SIN VIOLENCIA'
 'ROBO A PASAJERO A BORDO DE TAXI CON VIOLENCIA' 'HECHO NO DELICTIVO'
 'ROBO A PASAJERO A BORDO DEL METRO CON Y SIN VIOLENCIA'
 'ROBO A REPARTIDOR CON Y SIN VIOLENCIA'
 'ROBO DE VEHÍCULO CON Y SIN VIOLENCIA' 'ROBO A NEGOCIO CON VIOLENCIA'
 'HOMICIDIO DOLOSO' 'VIOLACIÓN' 'ROBO A CASA HABITACIÓN CON VIOLENCIA'
 'ROBO A PASAJERO A BORDO DE MICROBUS CON Y SIN VIOLENCIA'
 'LESIONES DOLOSAS POR DISPARO DE ARMA DE FUEGO'
 'ROBO A TRANSPORTISTA CON Y SIN VIOLENCIA'
 'ROBO A CUENTAHABIENTE SALIENDO DEL CAJERO CON VIOLENCIA' 'SECUESTRO']


# Clasificación de Delitos

Como podemos ver, hay 16 categorías de delitos. Podemos, por simplicidad, clasificarlas de la siguiente forma:

# Delitos Nivel 1 (Delitos Menores):
*Se definen como delitos sin repercusiones serias. 

    -Delitos de bajo impacto
    -Hecho no delictivo

# Delitos Nivel 2 (Delitos Nivel Medio):
*Se definen como delitos serios, que impactan el bienestar de la comunidad, pero que no son tan graves como 
los del Nivel 3 (homicidio, violación, etc.) Los delitos del Nivel 2 son los delitos 'del día a día'

    -Robo a transeúnte en vía pública con y sin violencia
    -Robo a pasajero a bordo de taxi con violencia
    -Robo a pasajero a bordo del metro con y sin violencia
    -Robo a repartidor con y sin violencia
    -Robo a pasajero a bordo de microbus con y sin violencia
    -Robo a transportista con y sin violencia
    -Robo a cuentahabiente saliendo del cajero con violencia

# Delitos Nivel 3 (Delitos Graves):
*En esta categoría entran los delitos cuya magnitud no puede ser subestimada o que tienen un impacto serio en 
el nivel de seguridad de la comunidad. Por ejemplo, en esta categoría entra 'robo a casa habitación con violencia' porque me parece mucho más grave que entren a tu casa y destruyan tu seguridad a que te roben en el transporte público. Obviamente ambos son hechos terribles, pero es más grave el daño material y psicólogico cuando irrumpen en tu hogar. Igualmente, es más grave que te roben tu coche a que te roben tu laptop en el metro, por ejemplo. Los delitos nivel 3 son los que más ponderación tendrán al determinar el nivel de seguridad de una colonia. 

    -Secuestro
    -Homicidio Doloso
    -Violación
    -Lesiones dolosas por disparo de arma de fuego
    -Robo a negocio con violencia
    -Robo de vehículo con y sin violencia
    -Robo a casa habitación con violencia.

Cabe aclarar que por hacer esta clasificación de ninguna manera quiero menospreciar el impacto 
en una persona que pueda tener un delito; simplemente necesitaba crear una métrica para clasificar
los delitos de *alguna* manera. 

In [6]:
#Vamos a añadir una columna con el ranking. 

clasif = {
    'DELITO DE BAJO IMPACTO': 1, 'HECHO NO DELICTIVO':1,   #delitos nivel 1 
          
    'ROBO A TRANSEUNTE EN VÍA PÚBLICA CON Y SIN VIOLENCIA': 2,  #delitos nivel 2
    'ROBO A PASAJERO A BORDO DE TAXI CON VIOLENCIA': 2, 
    'ROBO A PASAJERO A BORDO DEL METRO CON Y SIN VIOLENCIA': 2, 
    'ROBO A REPARTIDOR CON Y SIN VIOLENCIA': 2, 
    'ROBO A PASAJERO A BORDO DE MICROBUS CON Y SIN VIOLENCIA': 2,
    'ROBO A TRANSPORTISTA CON Y SIN VIOLENCIA': 2,
    'ROBO A CUENTAHABIENTE SALIENDO DEL CAJERO CON VIOLENCIA': 2,
          
    'SECUESTRO': 3,
    'HOMICIDIO DOLOSO': 3,
    'VIOLACIÓN': 3,
    'LESIONES DOLOSAS POR DISPARO DE ARMA DE FUEGO': 3,
    'ROBO A NEGOCIO CON VIOLENCIA': 3,
    'ROBO DE VEHÍCULO CON Y SIN VIOLENCIA': 3,
    'ROBO A CASA HABITACIÓN CON VIOLENCIA': 3
    }


data['score_delito'] = data.categoria_delito.map(clasif)

In [7]:
data.score_delito.unique()

array([1, 2, 3])

# Segunda Clasificación de Delitos: Clasificación Binaria. 

La idea en este caso es muy simple. Me interesa tener una categoría por colonia en donde se indique 
qué tan seguro es su transporte público. Entonces voy a agregar una columna donde haya un 1 
si el delito fue 'robo a pasajero a bordo del metro con y sin violencia' o 'robo a pasajero a bordo de microbus
con y sin violencia', y 0 en otro caso. 

In [8]:
data['score_transporte'] = np.where((data.categoria_delito == 'ROBO A PASAJERO A BORDO DEL METRO CON Y SIN VIOLENCIA')|(data.categoria_delito == 'ROBO A PASAJERO A BORDO DE MICROBUS CON Y SIN VIOLENCIA'), 1, 0)

In [9]:
data.score_transporte.unique()

array([0, 1])

In [10]:
def square_num(lst):
    sq = [int(x)**2 for x in lst]
    return sum(sq)

In [11]:
#Ahora sí, vamos analizando por colonia qué onda. 

ranking_colonias = pd.pivot_table(data[['colonia_hechos', 'score_delito']], index = data.colonia_hechos,
                                                                  aggfunc=square_num)

# Nota:

    -Es importante notar que yo ponderé por el cuadrado del score del delito. Esto significa que delitos nivel 3, 
    por ejemplo, suman 9 al score_delito, mientras que delitos nivel 1 solo suman 1. Delitos nivel 2, por supuesto, suman 4. Hice esto para que las colonias con los delitos más graves tengan un score mucho más alto que las colonias con delitos menores. Como se verá más adelante, aún hubo que hacer muchos ajustes para llegar a una métrica razonable de clasificación. 

In [12]:
ranking_colonias = ranking_colonias.reset_index()
ranking_colonias.sort_values(by=['score_delito'], ascending=False).head(15)

,colonia_hechos,score_delito
312,CENTRO,45112
415,DOCTORES,22313
406,DEL VALLE CENTRO,17651
1201,ROMA NORTE,16122
55,AGRÍCOLA ORIENTAL,14886
955,NARVARTE,13517
942,MORELOS,13029
262,BUENAVISTA,12371
601,GUERRERO,12047
56,AGRÍCOLA PANTITLAN,10870


Este fue el primer paso, pero no es suficiente. Si prestamos con atención a las 15 colonias más 
inseguras, en realidad llama la atención que ahí está Polanco y la colonia Juárez, colonias famosas por su 
lujo y comodidad, pero no por su inseguridad. Lo que tenemos que hacer es dividir este score por el número
de delitos que hubo en cada colonia para obtener una métrica más adecuada. 

In [13]:
colonias = list(data.colonia_hechos.unique())

In [14]:
#colonias

In [15]:
dictio = {}
for colonia in colonias:
    
    dictio[colonia] = len(data[data.colonia_hechos==colonia]) 

In [16]:
#dictio

In [17]:
ranking_colonias['numero_delitos'] = ranking_colonias.colonia_hechos.map(dictio)

In [18]:
ranking_colonias['score_ponderado'] = (ranking_colonias.score_delito) / (ranking_colonias.numero_delitos)

In [19]:
"""
#Ignorar estos cálculos, estaba haciendo prueba

q1 = ranking_colonias.score_ponderado.quantile(.05)
q2 = ranking_colonias.score_ponderado.quantile(.25)
q3 = ranking_colonias.score_ponderado.quantile(.50)
q4 = ranking_colonias.score_ponderado.quantile(.75)


labels = [1,2,3,4]

ranking_colonias['nivel_peligro'] = pd.qcut(ranking_colonias.score_ponderado, [0, .25, .5, .75, 1.],
                                                  labels = labels)
                                                  
"""

In [30]:
ranking_colonias.sort_values(by=['score_ponderado'], ascending=False).head(15)
#Como podemos ver, ya no aparece Polanco ni la colonia Juárez. 
#Ahora hay otro problema: aparecen colonias con muy pocos crímenes como "muy inseguras" aunque en realidad 
#solo han habido un par de crímenes por esa colonia en los últimos años. 

,colonia_hechos,score_delito,numero_delitos,score_ponderado
1013,PARQUE ECOLOGICO DE XOCHIMILCO,1064,212,5.018868
1550,VALLE VERDE,10,2,5.000000
1481,TLACAELEL 2,65,14,4.642857
1067,PISTA OLIMPICA VIRGILIO URIBE (CANAL DE CUEMANCO),474,103,4.601942
26,3 DE MAYO,94,22,4.272727
1214,SAN AGUSTÍN (DEL PUEBLO DE SAN ANDRÉS MIXQUIC),21,5,4.200000
1254,SAN ISIDRO TULYEHUALCO,235,59,3.983051
1451,TEJOMULCO EL ALTO,54,14,3.857143
466,EL OLIVO (SAN LORENZO),26,7,3.714286
1495,TLAXCALTENCO LA MESA,22,6,3.666667


In [21]:
#Lo que necesitamos es un criterio que combine ambas cosas. 
#Veamos el rango de score ponderado 

ranking_colonias.score_ponderado.describe()

#Podemos ver que un score arriba de 2.16 ya te pone en el top 25% más inseguro. 
#Un score entre 1 y 1.56 es normal. 

count    1627.000000
mean        1.871011
std         0.495417
min         1.000000
25%         1.562994
50%         1.833333
75%         2.160081
max         5.018868
Name: score_ponderado, dtype: float64

In [22]:
#Ahora veamos la columna de numero_delitos
ranking_colonias.numero_delitos.describe()

#Vemos que en promedio han habido 476 delitos por colonia, pero la varianza es enorme. 
#La colonia con más delitos tiene casi 30,000 delitos en los últimos años, mientras que el 25% de colonias
#más seguras tiene tan solo 60 delitos desde el 2014, prácticamente un delito y medio por mes. 

count     1627.000000
mean       476.442532
std       1189.128758
min          1.000000
25%         61.000000
50%        186.000000
75%        490.000000
max      28708.000000
Name: numero_delitos, dtype: float64

In [23]:
"""


q1 = ranking_colonias.numero_delitos.quantile(.05)
q2 = ranking_colonias.numero_delitos.quantile(.25)
q3 = ranking_colonias.numero_delitos.quantile(.50)
q4 = ranking_colonias.numero_delitos.quantile(.75)


labels = [1, 2, 3, 4]

ranking_colonias['nivel_frecuencia_delitos'] = pd.qcut(ranking_colonias.numero_delitos, [0, .25, .5, .75, 1.],
                                                  labels = labels)
"""

"\n\n\nq1 = ranking_colonias.numero_delitos.quantile(.05)\nq2 = ranking_colonias.numero_delitos.quantile(.25)\nq3 = ranking_colonias.numero_delitos.quantile(.50)\nq4 = ranking_colonias.numero_delitos.quantile(.75)\n\n\nlabels = [1, 2, 3, 4]\n\nranking_colonias['nivel_frecuencia_delitos'] = pd.qcut(ranking_colonias.numero_delitos, [0, .25, .5, .75, 1.],\n                                                  labels = labels)\n"

In [31]:
ranking_colonias.sort_values(by=['numero_delitos'], ascending=False).head(15)

,colonia_hechos,score_delito,numero_delitos,score_ponderado
312,CENTRO,45112,28708,1.571409
415,DOCTORES,22313,16577,1.346022
406,DEL VALLE CENTRO,17651,12628,1.397767
1201,ROMA NORTE,16122,10390,1.551684
955,NARVARTE,13517,9306,1.452504
262,BUENAVISTA,12371,7817,1.582576
690,JUÁREZ,10182,7171,1.419886
1078,POLANCO,9527,7122,1.337686
601,GUERRERO,12047,6780,1.776844
942,MORELOS,13029,6759,1.927652


In [26]:
ranking_colonias.dtypes

colonia_hechos       object
score_delito          int64
numero_delitos        int64
score_ponderado     float64
nivel_peligro      category
dtype: object

# Criterio Final de Clasificación

Recapitulemos brevemente lo que hemos hecho:

    1. Clasificamos los delitos según su gravedad en Nivel 1, Nivel 2 y Nivel 3.
    
    2. Creamos un nuevo DataFrame en donde tenemos colonias en una columna y su score en otra columna,
    donde el score es igual a la suma del cuadrado del nivel de gravedad de sus delitos. (Por ejemplo, una colonia con dos delitos nivel uno, un delito nivel dos y tres delitos nivel 3 tiene un score de: 
    (1)^2 + (1)^2 + (2)^2 + (3)^2 + (3)^2 + (3)^2 = 33. 
    
    3. Descubrimos que eso no nos da un buen ranking de colonias por seguridad, dado que no estamos tomando
    en cuenta el número de delitos que ha habido. 
    
    4. Obtuvimos una nueva métrica que es el 'score ponderado', donde simplemente se divide al score de delitos
    por el número de delitos. 
    
    5. Descubrimos que esta métrica nos ayuda a clarificar un poco las cosas pero no resuelve una última dificultad: no toma en cuenta el número *absoluto de delitos. Podemos tener una colonia con un solo delito nivel 3 y esa colonia tendría un score ponderado de 9, superando a las otras 1600 colonias de la CDMX, aunque solo haya 
    ocurrido un delito en los últimos 5 años (la base de datos es desde el 2014). 
    
    6. Veremos cómo solucionar este problema a continuación.

In [32]:
"""
La solución que se me ocurre es la siguiente:

Dado que la respuesta claramente está considerando tanto el número de delitos como la gravedad de los delitos,
lo que haré es normalizar la columna de score ponderado (en otra columna) y luego normalizar la columna de número
de delitos, para así tener dos columnas extra que van del 0-1, en donde 1 reflejará mayor peligro. 
Con esto obtendré una tercera columna final donde se obtendrá la calificación de la colonia. Esto simplemente
será un promedio ponderado de ambos números (más detalles abajo).

Después solo es cosa de dividirlo entre 1 si queremos que las más peligrosas tengan un score bajo (se puede 
pensar como que tienen bajo porcentaje de seguridad). Creo que para el usuario final va a ser más fácil verlo como
porcentaje de seguridad, donde una columna con 100% de porcentaje da mucha confianza y otra con 0% da poca.


"""


'\nLa solución que se me ocurre es la siguiente:\n\nDado que la respuesta claramente está considerando tanto el número de delitos como la gravedad de los delitos,\nlo que haré es normalizar la columna de score ponderado (en otra columna) y luego normalizar la columna de número\nde delitos, para así tener dos columnas extra que van del 0-1, en donde 1 reflejará mayor peligro. \nCon esto obtendré una tercera columna final donde se obtendrá la calificación de la colonia. Esto simplemente\nserá el promedio de ambos números. \n\nDespués solo es cosa de dividirlo entre 1 si queremos que las más peligrosas tengan un score bajo (se puede \npensar como que tienen bajo porcentaje de seguridad). Creo que para el usuario final va a ser más fácil verlo como\nporcentaje de seguridad, donde una columna con 100% de porcentaje da mucha confianza y otra con 0% da poca.\n\n\n'

In [39]:
#calculemos la columna que normaliza el score de delitos
from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [40]:
#normalizamos el score ponderado, que más bien debería llamarse score promedio
#recordemos que este score es el score promedio de los delitos de la colonia. 
ranking_colonias['score_ponderado_normalizado'] = MinMaxScaler().fit_transform(np.array(ranking_colonias.score_ponderado).reshape(-1,1))

,colonia_hechos,score_delito,numero_delitos,score_ponderado,score_ponderado_normalizado
0,10 DE ABRIL,144,84,1.714286,0.177733
1,10 DE MAYO,398,272,1.463235,0.115265
2,12 DE DICIEMBRE,190,99,1.919192,0.228719
3,15 DE AGOSTO,584,300,1.946667,0.235556
4,16 DE SEPTIEMBRE,303,186,1.629032,0.156520


In [45]:
#tiempo de normalizar el número de delitos
ranking_colonias['num_delitos_normalizado'] = MinMaxScaler().fit_transform(np.array(ranking_colonias.numero_delitos).reshape(-1,1))

In [46]:
ranking_colonias.sort_values(by=['num_delitos_normalizado', 'score_ponderado_normalizado'], ascending=False).head()

,colonia_hechos,score_delito,numero_delitos,score_ponderado,score_ponderado_normalizado,num_delitos_normalizado
312,CENTRO,45112,28708,1.571409,0.142181,1.000000
415,DOCTORES,22313,16577,1.346022,0.086099,0.577420
406,DEL VALLE CENTRO,17651,12628,1.397767,0.098975,0.439858
1201,ROMA NORTE,16122,10390,1.551684,0.137274,0.361898
955,NARVARTE,13517,9306,1.452504,0.112595,0.324137


In [63]:
#Los pesos de .7 y .3 evitan que colonias que están hasta arriba de la distribución en score, pero que realmente
#es porque tuvieron solo 10 crímenes o algo así, salgan ahí.
ranking_colonias['promedio_v1'] = (ranking_colonias.score_ponderado_normalizado)*.3 + (ranking_colonias.num_delitos_normalizado)*.70

In [64]:
#Vemos con un head(15) y tail(15) que este procedimiento me los ordena bastante bien. Voy a guardar esto como
#un nuevo data frame.
ranking_colonias.sort_values(by=['promedio_v1'], ascending=False).tail(15)

,colonia_hechos,score_delito,numero_delitos,score_ponderado,score_ponderado_normalizado,num_delitos_normalizado,promedio_v1
1200,RIO DE GUADALUPE FOVISSSTE,2,2,1.0,0.0,0.000035,0.000024
636,INFONAVIT,2,2,1.0,0.0,0.000035,0.000024
1594,VISTA HERMOSA I,2,2,1.0,0.0,0.000035,0.000024
282,CAMPAMENTO LA ESMERALDA,2,2,1.0,0.0,0.000035,0.000024
459,EL MIRADOR DEL PUEBLO TETELPAN,2,2,1.0,0.0,0.000035,0.000024
891,MALVINAS SEGUNDO,2,2,1.0,0.0,0.000035,0.000024
238,BELEN DE LAS FLORES SECCION TURBINA,1,1,1.0,0.0,0.000000,0.000000
921,MILPA LOS CERROS,1,1,1.0,0.0,0.000000,0.000000
890,MALVINAS PRIMERO,1,1,1.0,0.0,0.000000,0.000000
882,LÁZARO TATA,1,1,1.0,0.0,0.000000,0.000000


In [65]:
colonias_rankeado = ranking_colonias.sort_values(by=['promedio_v1'], ascending=False).copy()

In [66]:
colonias_rankeado = colonias_rankeado.reset_index()

In [68]:
colonias_rankeado.head(20)

,index,colonia_hechos,score_delito,numero_delitos,score_ponderado,score_ponderado_normalizado,num_delitos_normalizado,promedio_v1
0,312,CENTRO,45112,28708,1.571409,0.142181,1.000000,0.742654
1,415,DOCTORES,22313,16577,1.346022,0.086099,0.577420,0.430024
2,406,DEL VALLE CENTRO,17651,12628,1.397767,0.098975,0.439858,0.337593
3,1013,PARQUE ECOLOGICO DE XOCHIMILCO,1064,212,5.018868,1.000000,0.007350,0.305145
4,1550,VALLE VERDE,10,2,5.000000,0.995305,0.000035,0.298616
5,1201,ROMA NORTE,16122,10390,1.551684,0.137274,0.361898,0.294511
6,1481,TLACAELEL 2,65,14,4.642857,0.906439,0.000453,0.272249
7,1067,PISTA OLIMPICA VIRGILIO URIBE (CANAL DE CUEMANCO),474,103,4.601942,0.896258,0.003553,0.271365
8,955,NARVARTE,13517,9306,1.452504,0.112595,0.324137,0.260674
9,55,AGRÍCOLA ORIENTAL,14886,6654,2.237151,0.307836,0.231755,0.254579


Ahora viene lo bello. Para el usuario será más intuitivo y fácil ver un score de "seguridad" del 1-100%.
Por ejemplo, que vea que la colonia doctores tendría un score de (1-.43) = 57% seguro, no muy segura realmente. 
La colonia juárez tendría un score del 80%, por ejemplo. Lo que necesito hacer es solo agregar una columna que sea 1 - promedio_v1 y luego rankear por eso para tener las colonias más seguras!

In [72]:
colonias_rankeado['safety_degree'] = colonias_rankeado.promedio_v1.apply(lambda x: 1-x)

In [73]:
colonias_rankeado = colonias_rankeado.sort_values(by=['safety_degree'], ascending=False)

In [74]:
colonias_rankeado.head(10) #las 10 colonias más seguras han tenido 1 crimen en los últimos 10 años.

,index,colonia_hechos,score_delito,numero_delitos,score_ponderado,score_ponderado_normalizado,num_delitos_normalizado,promedio_v1,safety_degree
1626,287,CAMPO DE TIRO LOS GAMITOS,1,1,1.0,0.0,0.000000,0.000000,1.000000
1625,1424,SECTOR MARINA,1,1,1.0,0.0,0.000000,0.000000,1.000000
1624,1281,SAN JUAN TEPENAHUAC,1,1,1.0,0.0,0.000000,0.000000,1.000000
1623,1574,VILLA DE LA NORIA,1,1,1.0,0.0,0.000000,0.000000,1.000000
1622,800,LOMA BONITA,1,1,1.0,0.0,0.000000,0.000000,1.000000
1621,882,LÁZARO TATA,1,1,1.0,0.0,0.000000,0.000000,1.000000
1620,890,MALVINAS PRIMERO,1,1,1.0,0.0,0.000000,0.000000,1.000000
1619,921,MILPA LOS CERROS,1,1,1.0,0.0,0.000000,0.000000,1.000000
1618,238,BELEN DE LAS FLORES SECCION TURBINA,1,1,1.0,0.0,0.000000,0.000000,1.000000
1610,1006,PARAJE DEL CABALLITO,2,2,1.0,0.0,0.000035,0.000024,0.999976


In [75]:
colonias_rankeado.tail(10) #las 10 colonias más peligrosa

,index,colonia_hechos,score_delito,numero_delitos,score_ponderado,score_ponderado_normalizado,num_delitos_normalizado,promedio_v1,safety_degree
9,55,AGRÍCOLA ORIENTAL,14886,6654,2.237151,0.307836,0.231755,0.254579,0.745421
8,955,NARVARTE,13517,9306,1.452504,0.112595,0.324137,0.260674,0.739326
7,1067,PISTA OLIMPICA VIRGILIO URIBE (CANAL DE CUEMANCO),474,103,4.601942,0.896258,0.003553,0.271365,0.728635
6,1481,TLACAELEL 2,65,14,4.642857,0.906439,0.000453,0.272249,0.727751
5,1201,ROMA NORTE,16122,10390,1.551684,0.137274,0.361898,0.294511,0.705489
4,1550,VALLE VERDE,10,2,5.000000,0.995305,0.000035,0.298616,0.701384
3,1013,PARQUE ECOLOGICO DE XOCHIMILCO,1064,212,5.018868,1.000000,0.007350,0.305145,0.694855
2,406,DEL VALLE CENTRO,17651,12628,1.397767,0.098975,0.439858,0.337593,0.662407
1,415,DOCTORES,22313,16577,1.346022,0.086099,0.577420,0.430024,0.569976
0,312,CENTRO,45112,28708,1.571409,0.142181,1.000000,0.742654,0.257346


In [77]:
colonias_rankeado.colonia_hechos.describe()

count                     1627
unique                    1627
top       BOSQUES DEL PEDREGAL
freq                         1
Name: colonia_hechos, dtype: object

# SOLUCIÓN FINAL

## Solo queda limpiar un poco esta base para quitar las columnas que ya no me sirven, poner el safety degree con dos decimales nada más y después subirla a Mongo.

In [97]:
colonias_rankeado = colonias_rankeado.drop(columns=['score_delito', 'numero_delitos', 'score_ponderado',
                                                   'score_ponderado_normalizado', 'num_delitos_normalizado',
                                                   'promedio_v1'])

In [99]:
colonias_rankeado.safety_degree = colonias_rankeado.safety_degree.apply(lambda x: round(x*100,2))


,index,colonia_hechos,safety_degree
4,1550,VALLE VERDE,70.14
3,1013,PARQUE ECOLOGICO DE XOCHIMILCO,69.49
2,406,DEL VALLE CENTRO,66.24
1,415,DOCTORES,57.00
0,312,CENTRO,25.73


In [100]:
colonias_rankeado = colonias_rankeado.reset_index()

In [105]:
colonias_rankeado = colonias_rankeado.drop(columns=['level_0'])
colonias_rankeado = colonias_rankeado.rename(columns={'index': 'id_colonia'})
colonias_rankeado.head()

,id_colonia,colonia_hechos,safety_degree
0,287,CAMPO DE TIRO LOS GAMITOS,100.0
1,1424,SECTOR MARINA,100.0
2,1281,SAN JUAN TEPENAHUAC,100.0
3,1574,VILLA DE LA NORIA,100.0
4,800,LOMA BONITA,100.0


In [106]:
colonias_rankeado['ranking'] = [i for i in range(1, len(colonias_rankeado)+1)]
#queremos tener un ranking para Mongo. 

In [107]:
colonias_rankeado.head()

,id_colonia,colonia_hechos,safety_degree,ranking
0,287,CAMPO DE TIRO LOS GAMITOS,100.0,1
1,1424,SECTOR MARINA,100.0,2
2,1281,SAN JUAN TEPENAHUAC,100.0,3
3,1574,VILLA DE LA NORIA,100.0,4
4,800,LOMA BONITA,100.0,5


In [108]:
colonias_rankeadas_final = colonias_rankeado.to_csv('colonias_rankeadas.csv')

In [110]:
colonias_rankeado.safety_degree = colonias_rankeado.safety_degree.apply(lambda x: str(x)+ " %")

In [113]:
colonias_rankeado = colonias_rankeado.drop(columns=['id_colonia'])

In [115]:
colonias_rankeado.head(10)

,colonia_hechos,safety_degree,ranking
0,CAMPO DE TIRO LOS GAMITOS,100.0 %,1
1,SECTOR MARINA,100.0 %,2
2,SAN JUAN TEPENAHUAC,100.0 %,3
3,VILLA DE LA NORIA,100.0 %,4
4,LOMA BONITA,100.0 %,5
5,LÁZARO TATA,100.0 %,6
6,MALVINAS PRIMERO,100.0 %,7
7,MILPA LOS CERROS,100.0 %,8
8,BELEN DE LAS FLORES SECCION TURBINA,100.0 %,9
9,PARAJE DEL CABALLITO,100.0 %,10


In [116]:
colonias_rankeadas_porcentaje = colonias_rankeado.to_csv("colonias_rankeadas_pct.csv")